In [7]:
import time
import ray

In [8]:
@ray.remote
class Counter:
    def __init__(self):
        self.value = 0

    def increment(self):
        self.value += 1
        return self.value

    def get_counter(self):
        return self.value

# Specify required resources for an actor.
# @ray.remote(num_cpus=2, num_gpus=0.5)
# class Actor:
#     pass

In [12]:
# Create an actor from this class.
counter = Counter.remote()
obj_ref = counter.increment.remote()
print(type(obj_ref))
print(ray.get(obj_ref)) # result: 1

<class 'ray._raylet.ObjectRef'>
1


In [16]:
# Create ten Counter actors.
counters = [Counter.remote() for _ in range(10)]
print("counters is: ", counters)

# Increment each Counter once and get the results. These tasks all happen inparallel.
results = ray.get([c.increment.remote() for c in counters])
print("using counter call increment function: ", results)

# Increment the first Counter five times. These tasks are executed serially and share state.
results = ray.get([counters[0].increment.remote() for _ in range(5)])
print(results)

test = Counter.remote()
print(ray.get(test.get_counter.remote())) # result: 0
print(ray.get(counters[1].get_counter.remote())) # result: 1
print(ray.get(counters[0].get_counter.remote())) # result: 6

counters is:  [Actor(Counter, c1878944970166dbc5ea421d01000000), Actor(Counter, 32e850b2e78fa0b7862e7ca201000000), Actor(Counter, 80005d2f3c0cfbe8305a611301000000), Actor(Counter, 11a0776a961dde8f4fc0f2d201000000), Actor(Counter, 7e85c5e568ac32fa499d89d801000000), Actor(Counter, b67808955b5811fb3b2f108801000000), Actor(Counter, 4201855a0247ee437fd2a9d201000000), Actor(Counter, 33af3b7d94138fe1dedf5fca01000000), Actor(Counter, 530b5002b6ad9bb018f07e4301000000), Actor(Counter, c66fdbd2fef02221bf12385301000000)]
using counter call increment function:  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[2, 3, 4, 5, 6]
0
1
6


In [17]:
@ray.remote
def f(counter):
    for _ in range(10):
        time.sleep(0.1)
        counter.increment.remote()

In [22]:
counter = Counter.remote()

# Start some tasks that use the actor.
[f.remote(counter) for _ in range(3)] # list 中含三项，每一项对counter做10次递增

# Print the counter value.
for _ in range(10):
    time.sleep(0.1)
    print(ray.get(counter.get_counter.remote()))

10
30
30
30
30
30
30
30
30
30
